<a href="https://colab.research.google.com/github/Siqi-SN/2025-Fall---Language-Analytics/blob/main/Dict/final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Load the Drive helper
from google.colab import drive

# Below will prompt for authorization but it will make your google drive available (i.e., mount your drive).
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#find out where you are and move to correct location
import os #package for figuring out operating system

os.getcwd() #what is the current working directory

os.chdir("/content/drive/MyDrive/Colab Notebooks/Data/") #change directory

os.listdir() #data is there

['MOOC_reviews', 'Language_analytics_fall_25', 'SS_POS_tags.ipynb']

# Data Cleaning

delete reviews not in English

In [ ]:
df = pd.read_csv("MOOC_reviews/Coursera_reviews.csv")

In [ ]:
!pip install langdetect

In [ ]:
def detect_language_with_langdetect(line):
    from langdetect import detect_langs
    try:
        langs = detect_langs(line)
        for item in langs:
            # The first one returned is usually the one that has the highest probability
            return item.lang, item.prob
    except: return "err", 0.0

In [ ]:
# Filter for English reviews only
df_reviews_en = df[
    df['reviews'].apply(lambda x: detect_language_with_langdetect(x)[0] == 'en')
]

print(f"The total number of English Coursera reviews is: {len(df_reviews_en)}")

In [ ]:
# save the file
df_reviews_en.to_csv(r"coursera_reviews_en.csv",
                   index=False,
                   encoding='utf-8')
print("coursera_reviews_en has been saved as csv file")
# The print will not work bc I only copied and pasted the codes from the last assignment without running them. It takes hours to run...

filter reviews with more than 50 words

In [ ]:
import pandas as pd
import spacy

df_reviews = pd.read_csv("MOOC_reviews/coursera_reviews_en.csv")

nlp = spacy.load("en_core_web_sm")

In [ ]:
# calculate word count
def word_count(text):
    doc = nlp(str(text))
    words = [token.text for token in doc if not token.is_punct and not token.is_space]
    return len(words)

In [ ]:
# add coloum "word_count"
df_reviews['word_count'] = df_reviews['reviews'].apply(word_count)

# filter reviews with tokens >= 50
df_reviews = df_reviews[df_reviews['word_count'] >= 50].reset_index(drop=True)

print(f"The total number of English Coursera reviews with tokens >= 50 is: {len(df_reviews)}")

In [ ]:
df_reviews.to_csv(r"coursera_reviews_en_50.csv",
                   index=False,
                   encoding='utf-8')
print("coursera_reviews_en_50 has been saved as csv file")

Downsample the positive reviews

In [ ]:
df_reviews = pd.read_csv("MOOC_reviews/coursera_reviews_en_50.csv")

# create label: POS (rating 4/5) and NEG (rating 1/2)
def assign_label(rating):
    if rating in [4, 5]:
        return "POS"
    elif rating in [1, 2]:
        return "NEG"
    else:
        return "Neutral"

df_reviews["label"] = df_reviews["rating"].apply(assign_label)

print(df_reviews[['rating', 'label']].value_counts())

pos_reviews = df_reviews[df_reviews['label']=="POS"].copy()
neg_reviews = df_reviews[df_reviews['label']=="NEG"].copy()

In [ ]:
# downsample POS and create an equal subsample of NEG from Gemini
target_total = len(neg_reviews)

# calculate the proportions for course_id then rating
proportions = pos_reviews.groupby(['course_id', 'rating']).size().reset_index(name='count')
proportions['proportion'] = proportions['count'] / len(pos_reviews)

# stratified downsample
sample_sizes = (proportions['proportion'] * target_total).astype(int)

pos_down = pos_reviews.groupby(['course_id', 'rating'], group_keys=False).apply(
    lambda x: x.sample(n=sample_sizes[(proportions['course_id'] == x.name[0]) & (proportions['rating'] == x.name[1])].iloc[0], random_state=42)
)

In [ ]:
print(proportions.head())
print(pos_down[['rating']].value_counts())

In [ ]:
# combine POS + NEG
balanced_reviews = pd.concat([pos_down, neg_reviews], axis=0).reset_index(drop=True)

# Results
print(balanced_reviews[['rating', 'label']].value_counts())
#print(balanced_reviews.head())

In [ ]:
balanced_reviews.to_csv(r"balanced_reviews.csv",
                   index=False,
                   encoding='utf-8')
print("balanced_reviews has been saved as csv file")

remove courses with only 1 review left

In [ ]:
import pandas as pd

df = pd.read_csv("MOOC_reviews/balanced_reviews.csv")

#print(df.head())
#print(df[['rating', 'label']].value_counts())
#print(f"The total number of balanced reviews is: {len(df)}")


In [ ]:
# the count of reviews per course
course_counts = df[['course_id']].value_counts()
#print(course_counts)

# the number of courses with only 1 review left
course_counts[course_counts == 1].count()


np.int64(30)

In [ ]:
# filter
courses_with_single_review = course_counts[course_counts == 1].index.tolist()
#print(courses_with_single_review)

# flat the list of tuples
courses_to_remove = [course[0] for course in courses_with_single_review ]
print(courses_to_remove)

df_filtered = df[~df['course_id'].isin(courses_to_remove)].reset_index(drop=True)

#df_filtered

print(f"The total number of reviews after filtering is: {len(df_filtered)}")
print(f"The number of unique courses after filtering is: {df_filtered['course_id'].nunique()}")

['legal-tech-startups', 'excellence-operationnelle', 'business-model', 'oralcavity', 'mathematics-and-python', 'getting-started-with-tensor-flow2', 'humanrights', 'foundations-of-mindfulness', 'gestionempresarialpyme', 'trunk-anatomy', 'financial-planning', 'introduction-to-data-analytics', 'health-behavior-change', 'introduction-to-cloud', 'introduction-statistics-data-analysis-public-health', 'international-taxation', 'digital-advertising-landscape', 'django-database-web-apps', 'devops-culture-and-mindset', 'google-cloud-product-fundamentals', 'c-plus-plus-white', 'high-stakes-leadership', 'everyday-excel-part-2', 'corporate-commercial-law-part1', 'data-science-k-means-clustering-python', 'arbitration-international-disputes', 'strategic-business-management-microeconomics', 'contracts-1', 'sustainable-finance', 'writing-editing-words']
The total number of reviews after filtering is: 19461
The number of unique courses after filtering is: 512


In [ ]:
# Calculate and print the average number of reviews per course
reviews_per_course = df_filtered.groupby('course_id').size()

sd_reviews_per_course = reviews_per_course.std()

print(f"The average number of reviews per course is: {average_reviews_per_course}")
print(f"The standard deviation of the number of reviews per course is: {sd_reviews_per_course}")

df_filtered[['label']].value_counts()

The average number of reviews per course is: 38.009765625
The standard deviation of the number of reviews per course is: 82.59199876870835


,count
label,
NEG,9744
POS,9717


final dataset

In [ ]:
df_filtered.to_csv(r"MOOC_reviews/balanced_review_final.csv",
                   index=False,
                   encoding='utf-8')
print("it has been saved as csv file")

it has been saved as csv file


# Sentiment Analysis




In [ ]:
import pandas as pd
import spacy

df = pd.read_csv("MOOC_reviews/balanced_review_final.csv")

nlp = spacy.load("en_core_web_sm")

## Descriptive Analysis

In [ ]:
sentence_word_counts = []

for doc in review_docs:
    for sent in doc.sents:
        word_count = 0
        for token in sent:
            # Check if the token is alphanumeric and not punctuation or space
            if token.is_alpha and not token.is_punct and not token.is_space:
                word_count += 1
        sentence_word_counts.append(word_count)


In [ ]:
#spaCy texts

review_docs = list(nlp.pipe(df.reviews))

## SA starts here

In [ ]:
# safe divide function to stop zero counts for words from causing problems
# some texts may have zero words after pre-processing

def safe_divide(a, b):
    if b != 0:
        return a/b
    else:
        return 0

read bing word lists

In [ ]:
bing_df = pd.read_csv('Language_analytics_fall_25/sentiment_datasets/Bing.csv')

bing_df

,word,sentiment
0,2-faces,negative
1,abnormal,negative
2,abolish,negative
3,abominable,negative
4,abominably,negative
...,...,...
6781,zealously,negative
6782,zenith,positive
6783,zest,positive
6784,zippy,positive


In [ ]:
# Data put into dictionaries

bing_dict = {"positive": [], "negative": []}

# iterate over the rows of bing_df
for _, row in bing_df.iterrows():
    sentiment = row['sentiment'].lower()  # get sentiment label
    word = row['word']                     # get the word
    bing_dict[sentiment].append(word)

# check the first 12 words in the negative category
print(bing_dict["negative"][0:12])

# see how many positive and negative words
print(f"There are {len(bing_dict['positive'])} positive words in the Bing lexicon.")
print(f"There are {len(bing_dict['negative'])} negative words in the Bing lexicon.")

['2-faces', 'abnormal', 'abolish', 'abominable', 'abominably', 'abominate', 'abomination', 'abort', 'aborted', 'aborts', 'abrade', 'abrasive']
There are 2005 positive words in the Bing lexicon.
There are 4781 negative words in the Bing lexicon.


Sentiment analysis

In [ ]:
nw_final = []
all_positives = []
all_negatives = []

all_adj_pos = []
all_adv_pos = []
all_verb_pos = []
all_adj_neg = []
all_adv_neg = []
all_verb_neg = []


for doc in review_docs:  # go through each document
    nw = 0  # count for number of words
    positive_count = 0
    negative_count = 0

    # New counts for specific POS/Sentiment (non-normalized)
    adj_pos_count = 0
    adv_pos_count = 0
    verb_pos_count = 0
    adj_neg_count = 0
    adv_neg_count = 0
    verb_neg_count = 0

    # Iterate through every token in the doc
    for token in doc:

        # 1. Count tokens for normalization
        if not token.is_punct and not token.is_space:
            nw += 1

        # 2. Only check sentiment for the POS tags you care about
        if token.pos_ not in {"ADJ", "ADV", "VERB"}:
            continue

        # 3. Check for negation
        is_negated = False

        # Check for direct negation child (e.g., "not good")
        for child in token.children:
            if child.dep_ == "neg":
                is_negated = True
                break

        # If not found, check head's children (e.g., "do not like")
        if not is_negated:
            for child in token.head.children:
                if child.dep_ == "neg":
                    is_negated = True
                    break

        # 4. Assign to the category
        lemma = token.lemma_
        pos_tag = token.pos_

        if lemma in bing_dict["positive"]:
            if is_negated:
                negative_count += 1
                # Increment specific negative POS count
                if pos_tag == "ADJ":
                    adj_neg_count += 1
                elif pos_tag == "ADV":
                    adv_neg_count += 1
                elif pos_tag == "VERB":
                    verb_neg_count += 1
            else:
                positive_count += 1
                # Increment specific positive POS count
                if pos_tag == "ADJ":
                    adj_pos_count += 1
                elif pos_tag == "ADV":
                    adv_pos_count += 1
                elif pos_tag == "VERB":
                    verb_pos_count += 1

        elif lemma in bing_dict["negative"]:
            if is_negated:
                positive_count += 1
                # Increment specific positive POS count (negated negative is positive)
                if pos_tag == "ADJ":
                    adj_pos_count += 1
                elif pos_tag == "ADV":
                    adv_pos_count += 1
                elif pos_tag == "VERB":
                    verb_pos_count += 1
            else:
                negative_count += 1
                # Increment specific negative POS count
                if pos_tag == "ADJ":
                    adj_neg_count += 1
                elif pos_tag == "ADV":
                    adv_neg_count += 1
                elif pos_tag == "VERB":
                    verb_neg_count += 1

    # 5. Save results (Normalized by total word count 'nw')
    nw_final.append(nw)
    all_positives.append(safe_divide(positive_count, nw))
    all_negatives.append(safe_divide(negative_count, nw))

    # Save new normalized POS/Sentiment results
    all_adj_pos.append(safe_divide(adj_pos_count, nw))
    all_adv_pos.append(safe_divide(adv_pos_count, nw))
    all_verb_pos.append(safe_divide(verb_pos_count, nw))
    all_adj_neg.append(safe_divide(adj_neg_count, nw))
    all_adv_neg.append(safe_divide(adv_neg_count, nw))
    all_verb_neg.append(safe_divide(verb_neg_count, nw))

In [ ]:
df['positive_count'] = all_positives
df['negative_count'] = all_negatives

df['word_count'] = nw_final

df['ADJ_POS'] = all_adj_pos
df['ADV_POS'] = all_adv_pos
df['VERB_POS'] = all_verb_pos

df['ADJ_NEG'] = all_adj_neg
df['ADV_NEG'] = all_adv_neg
df['VERB_NEG'] = all_verb_neg

print(df.head())

                                             reviews      reviewers  \
0  It's a great course! I learned a ton, and the ...  By SecureGive   
1  This course was well laid out for anyone just ...     By Sejal R   
2  Pretty decent introductory course to SQL.Pros:...   By Chutian Z   
3  It would have been much better if we could hav...   By Douglas C   
4  It's a very theoretical course and it gives a ...   By Valeria G   

   date_reviews  rating              course_id  word_count label  \
0  Jul 01, 2020       4            it-security          76   POS   
1  Aug 14, 2019       4       gcp-fundamentals          64   POS   
2  Jan 17, 2019       4   sql-for-data-science          87   POS   
3  Nov 18, 2019       4  introduction-genomics          52   POS   
4  Jul 15, 2017       4          arts-heritage         119   POS   

   positive_count  negative_count   ADJ_POS   ADV_POS  VERB_POS   ADJ_NEG  \
0        0.052632        0.000000  0.052632  0.000000  0.000000  0.000000   
1        0

In [ ]:
# POS reviews
pos_data = df[df['label'] == "POS"]

POS_pos_mean = pos_data['positive_count'].mean()
POS_pos_sd = pos_data['positive_count'].std()

POS_neg_mean = pos_data['negative_count'].mean()
POS_neg_sd = pos_data['negative_count'].std()

print(f"The mean of positive sentiment proportion is: {POS_pos_mean}")
print(f"The standard deviation of positive sentiment proportion is: {POS_pos_sd}")
print(f"The mean of negative sentiment proportion is: {POS_neg_mean}")
print(f"The standard deviation of negative sentiment proportion is: {POS_neg_sd}")

The mean of positive sentiment proportion is: 0.05946748765296836
The standard deviation of positive sentiment proportion is: 0.02875370099607204
The mean of negative sentiment proportion is: 0.0077687901800821465
The standard deviation of negative sentiment proportion is: 0.011420127528967598


In [ ]:
# NEG reviews
neg_data = df[df['label'] == "NEG"]

NEG_neg_mean = neg_data['negative_count'].mean()
NEG_neg_sd = neg_data['negative_count'].std()

NEG_pos_mean = neg_data['positive_count'].mean()
NEG_pos_sd = neg_data['positive_count'].std()

print(f"The mean of negative sentiment proportion is: {NEG_neg_mean}")
print(f"The standard deviation of negative sentiment proportion is: {NEG_neg_sd}")
print(f"The mean of positive sentiment proportion is: {NEG_pos_mean}")
print(f"The standard deviation of positive sentiment proportion is: {NEG_pos_sd}")


The mean of negative sentiment proportion is: 0.025750714717018206
The standard deviation of negative sentiment proportion is: 0.01862124913651422
The mean of positive sentiment proportion is: 0.024197264699997004
The standard deviation of positive sentiment proportion is: 0.017417994269484625


In [ ]:
# Save the dataframe as a CSV file
df.to_csv(r"final_project coursera_reviews.csv",
                   index=False,
                   encoding='utf-8')
print("it has been saved as csv file")

it has been saved as csv file


# Output


In [ ]:
!pip install nbconvert -q

In [ ]:
!apt-get install pandoc

Reading package lists...
Building dependency tree...
Reading state information...
pandoc is already the newest version (2.9.2.1-3ubuntu2).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.


In [ ]:
!apt-get install -y -qq texlive-xetex texlive-fonts-recommended texlive-plain-generic > /dev/null 2>&1

In [ ]:
# Convert the notebook to PDF /content/drive/MyDrive/Colab Notebooks/
!jupyter nbconvert --to pdf 'SS_POS_tags.ipynb' --output '/content/drive/MyDrive/Colab Notebooks/pos_tags.pdf'


[NbConvertApp] Converting notebook SS_POS_tags.ipynb to pdf
[NbConvertApp] Writing 76155 bytes to notebook.tex
[NbConvertApp] Building PDF
[NbConvertApp] Running xelatex 3 times: ['xelatex', 'notebook.tex', '-quiet']
[NbConvertApp] Running bibtex 1 time: ['bibtex', 'notebook']
[NbConvertApp] WARNING | bibtex had problems, most likely because there were no citations
[NbConvertApp] PDF successfully created
[NbConvertApp] Writing 85328 bytes to /content/drive/MyDrive/Colab Notebooks/pos_tags.pdf
